In [10]:
import cv2
import numpy as np
import glob

# --- 标定板参数（根据你的棋盘格规格修改） ---
CHECKERBOARD = (6, 8)  # 内角点个数 (行, 列)
square_size = 0.024    # 标定板方格边长，单位：米（可选，用于真实尺寸）

# --- 准备世界坐标点 (0,0,0), (1,0,0), (2,0,0)... ---
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp = objp * square_size

objpoints = []  # 3D 点
imgpoints = []  # 2D 点

# --- 读取标定图片路径，修改为你的文件夹 ---
images = glob.glob('/home/ubuntu/Desktop/project/sensor-calibration/samples/camera_calib_fisheye/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 寻找棋盘角点
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH +
                                             cv2.CALIB_CB_FAST_CHECK +
                                             cv2.CALIB_CB_NORMALIZE_IMAGE)

    if ret:
        # 亚像素角点精确化
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
        cv2.cornerSubPix(gray, corners, (3,3), (-1,-1), criteria)

        imgpoints.append(corners)
        objpoints.append(objp)

        # 可视化角点
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

if len(objpoints) < 1:
    print("未找到足够的棋盘格角点，标定失败！")
    exit()

# --- 标定鱼眼相机 ---
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = []
tvecs = []

img_shape = gray.shape[::-1]  # (width, height)

flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + \
        cv2.fisheye.CALIB_FIX_SKEW

rms, K, D, rvecs, tvecs = cv2.fisheye.calibrate(
    objectPoints=objpoints,
    imagePoints=imgpoints,
    image_size=img_shape,
    K=None,
    D=None,
    rvecs=rvecs,
    tvecs=tvecs,
    flags=flags,
    criteria=criteria
)

print("标定完成，RMS误差 =", rms)
print("内参矩阵 K:\n", K)
print("畸变系数 D:\n", D)



标定完成，RMS误差 = 2.0203099893251912
内参矩阵 K:
 [[519.78655084   0.         963.44116268]
 [  0.         520.85138735 525.51733617]
 [  0.           0.           1.        ]]
畸变系数 D:
 [[ 0.10779094]
 [-0.01369097]
 [-0.00873715]
 [ 0.00041709]]
